In [ ]:
from langchain_community.document_loaders import UnstructuredWordDocumentLoader
# from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import AzureOpenAIEmbeddings
import re
import uuid

from langchain_community.vectorstores import Chroma

from langchain_text_splitters import TextSplitter

embeddings = AzureOpenAIEmbeddings(
    azure_deployment="genai-text-embedding-3-small",   # your embedding model deployment name
    openai_api_key="a1b6648ba9c749e593296b291ed06bec",  # your Azure key
    azure_endpoint="https://spinnaker-genai.openai.azure.com/",
    openai_api_version="2024-12-01-preview"
)
# Fix 1: Correct the file path (use one of these options)
file_path = r"C:\Users\Kailash\Downloads\Master Bot Product Mapping.docx"  # Raw string


loader = UnstructuredWordDocumentLoader(file_path, mode='single')
documents = loader.load()

class DottedLineSplitter(TextSplitter):
    def split_text(self, text: str):
        chunks = re.split(r"(?:[-.]{5,})", text)
        return [chunk.strip() for chunk in chunks if chunk.strip()]
# split_docs = text_splitter.split_documents(documents)
splitter = DottedLineSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = splitter.split_documents(documents)

print(split_docs)

    

vectorstore = Chroma.from_documents(
    split_docs,
    embedding=embeddings,
    collection_name="routing_prompts",
    persist_directory="./chroma_db"  # This will save locally
)

print(f"Total chunks created: {len(split_docs)}\n")


[Document(metadata={'source': 'C:\\Users\\Kailash\\Downloads\\Master Bot Product Mapping.docx'}, page_content='Product Mapping\n\nSales Prophet\n\nurl : https://www.spinnakerhub.com/sales-prophet\n\ndescription: Sales Prophet™ is a comprehensive sales performance and planning platform designed to optimize market strategy, territory design, and profitability.\n\ncomponents / modules: [market opportunity generator (MOG), Sales Planning, Territory Design, Territory Design Tool, Broker Score Card, InsurIQ]\n\n1. MOG\n\nurl : [ https://www.spinnakerhub.com/sales-prophet/individual-life/market-overview, \n\nhttps://www.spinnakerhub.com/sales-prophet/annuity/market-overview,\n\nhttps://www.spinnakerhub.com/sales-prophet/worksite/market-overview,\n\nhttps://www.spinnakerhub.com/sales-prophet/employee-benefits/market-overview,\n\nhttps://www.spinnakerhub.com/sales-prophet/wealth-management/market-overview\n\n]\n\nDescription: Estimate granular market size and analyze client performance to devel

C:\Users\Kailash\AppData\Local\Temp\ipykernel_7116\2037673020.py:44: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [4]:
# azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
# openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
# azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
# openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION")

# # Same embedding configuration you used earlier
# embeddings = AzureOpenAIEmbeddings(
#     azure_deployment=azure_deployment,
#     openai_api_key=openai_api_key,
#     azure_endpoint=azure_endpoint,
#     openai_api_version=openai_api_version
# )

# Reconnect to your existing Chroma DB
vectorstore = Chroma(
    collection_name="routing_prompts",      # same name used earlier
    persist_directory="./chroma_db",        # same folder
    embedding_function=embeddings
)

query = "what is market premium"
results = vectorstore.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"\n--- RESULT {i+1} ---")
    print(f"Metadata: {doc.metadata}")
    print(f"Content preview:\n{doc.page_content}")



--- RESULT 1 ---
Metadata: {'source': 'C:\\Users\\Kailash\\Downloads\\Master Bot Product Mapping.docx'}
Content preview:
Product Mapping

Sales Prophet

url : https://www.spinnakerhub.com/sales-prophet

description: Sales Prophet™ is a comprehensive sales performance and planning platform designed to optimize market strategy, territory design, and profitability.

components / modules: [market opportunity generator (MOG), Sales Planning, Territory Design, Territory Design Tool, Broker Score Card, InsurIQ]

1. MOG

url : [ https://www.spinnakerhub.com/sales-prophet/individual-life/market-overview, 

https://www.spinnakerhub.com/sales-prophet/annuity/market-overview,

https://www.spinnakerhub.com/sales-prophet/worksite/market-overview,

https://www.spinnakerhub.com/sales-prophet/employee-benefits/market-overview,

https://www.spinnakerhub.com/sales-prophet/wealth-management/market-overview

]

Description: Estimate granular market size and analyze client performance to develop sales pote